In [1]:
from datasets import load_dataset

imdb = load_dataset("imdb")

In [2]:
imdb["test"][0]


{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator")

In [4]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True) #, max_length=1024)

In [5]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
import evaluate

accuracy = evaluate.load("accuracy")

In [8]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [9]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "google/electra-base-discriminator", num_labels=2, id2label=id2label, label2id=label2id
)

# for param in model.transformer.parameters():
#     param.requires_grad = False


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir="my_awesome_electra",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\Abstract\mambaforge\envs\sentenv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3126 [00:00<?, ?it/s]

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.2604, 'learning_rate': 1.6801023672424827e-05, 'epoch': 0.32}
{'loss': 0.1951, 'learning_rate': 1.3602047344849649e-05, 'epoch': 0.64}
{'loss': 0.1649, 'learning_rate': 1.0403071017274472e-05, 'epoch': 0.96}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.17841275036334991, 'eval_accuracy': 0.94188, 'eval_runtime': 307.838, 'eval_samples_per_second': 81.212, 'eval_steps_per_second': 5.077, 'epoch': 1.0}
{'loss': 0.1069, 'learning_rate': 7.204094689699297e-06, 'epoch': 1.28}
{'loss': 0.1056, 'learning_rate': 4.005118362124121e-06, 'epoch': 1.6}
{'loss': 0.1017, 'learning_rate': 8.061420345489445e-07, 'epoch': 1.92}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.16642089188098907, 'eval_accuracy': 0.95792, 'eval_runtime': 311.615, 'eval_samples_per_second': 80.227, 'eval_steps_per_second': 5.016, 'epoch': 2.0}
{'train_runtime': 2553.6139, 'train_samples_per_second': 19.58, 'train_steps_per_second': 1.224, 'train_loss': 0.1535106132370649, 'epoch': 2.0}


TrainOutput(global_step=3126, training_loss=0.1535106132370649, metrics={'train_runtime': 2553.6139, 'train_samples_per_second': 19.58, 'train_steps_per_second': 1.224, 'train_loss': 0.1535106132370649, 'epoch': 2.0})